# Import

In [2]:
import json
from jovis_model.configs.base import BaseConfig
from jovis_model.datasets.common import CommonDataModule

In [2]:
params = {
    "pkg": "klue",
    "task": "ynat",
    "data_dir": "/home/omnious/workspace/jovis/jovis-model/_db/klue/ynat-v1.1",
    "train_file_name": "ynat-v1.1_train.json",
    "dev_file_name": "ynat-v1.1_dev.json",
    "output_dir": "/home/omnious/workspace/jovis/jovis-model/outputs",
}

In [3]:
config = BaseConfig(
    **params
)

{}


In [4]:
dm = CommonDataModule(config)

BaseConfig(pkg='klue', task='ynat', data_dir='/home/omnious/workspace/jovis/jovis-model/_db/klue/ynat-v1.1', train_file_name='ynat-v1.1_train.json', dev_file_name='ynat-v1.1_dev.json', test_file_name=None, output_dir='/home/omnious/workspace/jovis/jovis-model/outputs', gpus=1, fp16=True, params=KLUEConfig(hf_name='klue/roberta-base', config_name='klue/roberta-base', tokenizer_name='klue/roberta-base', cache_dir=None, num_workers=4, train_batch_size=32, eval_batch_size=64, max_seq_length=128, num_sanity_val_steps=2, max_grad_norm=1.0, gradient_accumulation_steps=1, seed=42, metric='loss', patience=5, early_stopping_mode='max', encoder_layerdrop=None, decode_layerdrop=None, dropout=None, attention_dropout=None, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, warmup_steps=None, warmup_ratio=None, num_train_epochs=4, adafactor=True, verbose_step_count=100))

In [ ]:
config.to_json()

In [ ]:
class CommonTrainer:
    def __init__(self, task, dataset_params, train_params):
        

In [ ]:
data_path = "jovis_model/_db/klue/ynat-v1.1/ynat-v1.1_dev.json"
with open(data_path, "r", encoding="utf-8") as f:
    data_lst = json.load(f)

In [ ]:
data_lst[0]

In [ ]:
c = Config(
    **{
        "command": "train",
        "task": "ynat",
        "output_dir": "abcd",
        "train_config": {
            "m_name": "abcd",
            "config_name": "abcd",
            "tokenizer_name": "abcd",
            "cache_dir": "abcd",
            "encoder_layerdrop": 0.5,
            "decode_layerdrop": 0.5,
            "dropout": 0.5,
            "attention_dropout": 0.5,
            "adafactor": True
        }
    }
)

In [ ]:
c